In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob
from pylab import rcParams
import os
import warnings
import scipy.signal as signal
warnings.filterwarnings('ignore')
from datetime import datetime
rcParams['figure.figsize'] = 12,4
rcParams['savefig.dpi'] = 300
from detecta import detect_onset, detect_peaks
from tqdm import tqdm
logo_path = 'logo/'
now = datetime.now().strftime('%Y%m%d')

def lowpass_filter(data, sr, cut_off, order):
    nyq = 0.5 * sr
    b, a = signal.butter(order, cut_off/nyq, btype = 'low')
    lp_df = signal.filtfilt(b, a, data)
    return lp_df

In [2]:
day = 20240213
players = pd.read_excel(f'data/{day}/player information.xlsx', index_col='name')
file_list = sorted(glob(f'data/{day}//*csv'))
file_list

['data/20240213/uplift_data_export_20240213113022.csv',
 'data/20240213/uplift_data_export_20240213131339.csv',
 'data/20240213/uplift_data_export_20240213132832.csv',
 'data/20240213/uplift_data_export_20240213140631.csv',
 'data/20240213/uplift_data_export_20240213140719.csv',
 'data/20240213/uplift_data_export_20240213140726.csv',
 'data/20240213/uplift_data_export_20240213143034.csv',
 'data/20240213/uplift_data_export_20240213144038.csv',
 'data/20240213/uplift_data_export_20240213144852.csv',
 'data/20240213/uplift_data_export_20240213145603.csv',
 'data/20240213/uplift_data_export_20240213150413.csv',
 'data/20240213/uplift_data_export_20240213151249.csv',
 'data/20240213/uplift_data_export_20240213161158.csv',
 'data/20240213/uplift_data_export_20240213164704.csv',
 'data/20240213/uplift_data_export_20240214005949.csv',
 'data/20240213/uplift_data_export_20240214010153.csv',
 'data/20240213/uplift_data_export_20240214011427.csv',
 'data/20240213/uplift_data_export_2024021401230

In [3]:
all_sessions = []
for file in file_list:
    df = pd.read_csv(file)
    sessions = df.sessionid.unique()
    for session in tqdm(sessions):
        d = df[df['sessionid'] == session].sort_values('frame').reset_index(drop=True)
        side = d['handedness'][0]
        trial = str(session.split('_')[-1]).zfill(3)
        name = d['athlete_name'][0].replace(' ','')
        all_sessions.append(f"{name}_{trial}")
all_sessions

100%|██████████| 1/1 [00:00<00:00, 822.41it/s]


['n02_002',
 'n02_003',
 'n03_001',
 'n03_002',
 'n03_003',
 'n04_020',
 'n04_021',
 'n04_022',
 'n05_001',
 'n05_002',
 'n05_003',
 'n06_001',
 'n06_002',
 'n06_003',
 'n07_001',
 'n07_002',
 'n07_003',
 'n08_001',
 'n08_002',
 'n08_003',
 'n09_001',
 'n09_002',
 'n09_003',
 'n12_001',
 'n12_002',
 'n12_003',
 'n13_002',
 'n13_003',
 'n15_001',
 'n15_002',
 'n15_003',
 'n16_001',
 'n16_002',
 'n16_003',
 'n17_001',
 'n17_002',
 'n17_003',
 'n18_001',
 'n18_002',
 'n18_003',
 'n10_002',
 'n10_003',
 'n14_001',
 'n19_002',
 'n19_003',
 'n01_001',
 'n01_002',
 'n11_002']

In [4]:
wrong = {}
for file in file_list:
    print(file)
    df = pd.read_csv(file)
    sessions = df.sessionid.unique()
    for session in tqdm(sessions):
        d = df[df['sessionid'] == session].sort_values('frame').reset_index(drop=True)
        side = d['handedness'][0]
        trial = str(session.split('_')[-1]).zfill(3)
        name = d['athlete_name'][0].replace(' ','')
        weight, height, to_frame, kh_frame, fc_frame, bc_frame, ball_vel, launch_ang, ball_dis = players.loc[f"{name}_{trial}"]
        
        newcols = []
        for col in df.columns:
            if side == 'right':
                split_col = col.split('_')
                if split_col[0] == 'left':
                    split_col[0] = 'lead'
                    split_col = '_'.join(split_col)
                elif split_col[0] == 'right':
                    split_col[0] = 'rear'
                    split_col = '_'.join(split_col)
                elif 'trunk_lateral_flexion' in col:
                    split_col = 'trunk_lateral_flexion'
                else:
                    split_col = col
                
                newcols.append(split_col)
                
            elif side == 'left':
                split_col = col.split('_')
                if split_col[0] == 'left':
                    split_col[0] = 'rear'
                    split_col = '_'.join(split_col)
                elif split_col[0] == 'right':
                    split_col[0] = 'lead'
                    split_col = '_'.join(split_col)
                elif 'trunk_lateral_flexion' in col:
                    split_col = 'trunk_lateral_flexion'
                else:
                    split_col = col
            
                newcols.append(split_col)
               
        d.columns = newcols
        d = d.sort_values('frame').reset_index(drop=True)

        change_axis_col = []
        if side == 'left':
            for col in d.columns:
                if '3d' in col:
                    split_col = col.split('_')
                    if split_col[-1] == 'x':
                        split_col[-1] = 'z'
                    elif split_col[-1] == 'z':
                        split_col[-1] = 'x'
                    split_col = '_'.join(split_col)
                    change_axis_col.append(split_col)
                    
                else:
                    change_axis_col.append(col)
                    
            d.columns = change_axis_col
        # Sway (정강이 각도)
        y_axis = d['rear_knee_jc_3d_y'] - d['rear_ankle_jc_3d_y']
        z_axis = d['rear_knee_jc_3d_z'] - d['rear_ankle_jc_3d_z']
        shank_angle = np.arctan2(y_axis, z_axis)
        shank_angle = - (np.rad2deg(shank_angle) - 90)
        d['shank_angle'] = shank_angle
        
        # Loss of Space (골반 팔꿈치 거리)
        x1 = d['rear_hip_jc_3d_x']; y1 = d['rear_hip_jc_3d_z']
        x2 = d['lead_hip_jc_3d_x']; y2 = d['lead_hip_jc_3d_z']
        x3 = d['rear_elbow_jc_3d_x']; y3 = d['rear_elbow_jc_3d_z']

        area = abs((x1-x3) * (y2-y3) - (y1-y3) * (x2 - x3))
        AB = ( ((x1-x2)**2 + (y1-y2)**2) ** 0.5 )
        loss_space = area / AB
        d['hip_elbow_loss_space'] = loss_space
        
        # Loss of Space (어깨 리드 손의 거리)
    
        x2 = d['lead_shoulder_jc_3d_x']; y2 = d['lead_shoulder_jc_3d_y']; z2 = d['lead_shoulder_jc_3d_z']
        x3 = d['lead_wrist_jc_3d_x']; y3 = d['lead_wrist_jc_3d_y']; z3 = d['lead_wrist_jc_3d_z']
        
        D = np.sqrt((x2 - x3)**2 + (y2 - y3)**2 + (z2 - z3)**2)
        d['shoulder_hand_loss_space'] = D
        
        # Dead Hands (리드 암의 어깨와 손과의 수평 거리 (플레이트 방향)
        hand_shoulder_dis = d['lead_shoulder_jc_3d_z'] - d['lead_wrist_jc_3d_z']
        d['hand_shoulder_distance'] = hand_shoulder_dis
    
        if side == 'left':
            change_plus_minus = ['pelvis_rotational_velocity_with_respect_to_ground',
                                'trunk_rotational_velocity_with_respect_to_ground',
                                'lead_arm_rotational_velocity_with_respect_to_ground',
                                'trunk_lateral_flexion',
                                'trunk_twist_clockwise','shank_angle']
            d[change_plus_minus] = - d[change_plus_minus]
        # 골반 처음 움직이는 시점
        plot = False
        
        pel_rot_initial = [i for i in detect_onset(d['pelvis_rotational_velocity_with_respect_to_ground'], threshold=100, n_above=50, n_below =100, show=plot)[:,0] if kh_frame < i < bc_frame]
        if len(pel_rot_initial) == 1:
            pel_rot_initial = pel_rot_initial[0]
        else:
            wrong[file] = d
            
        # usecols = [
        # 'pelvis_rotational_velocity_with_respect_to_ground',
        # 'trunk_rotational_velocity_with_respect_to_ground', 
        # 'lead_arm_rotational_velocity_with_respect_to_ground',
        # 'lead_shoulder_external_rotation_velocity',
        # 'trunk_twist_clockwise','trunk_lateral_flexion',
        # 'lead_shoulder_external_rotation','lead_elbow_flexion',
        # 'rear_shoulder_adduction','lead_knee_extension',
        # 'lead_knee_extension_velocity','lead_hip_flexion_with_respect_to_trunk']

        # df = df[usecols]
        # df = df.iloc[to_frame:bc_frame + 60]
        t = np.arange(0,len(d)/240, 1/240)
        if len(t) == len(d):
            pass
        elif len(t) > len(d):
            t = t[:len(d)]
            
        
        
        if side == 'right':
            change_plus_minus = ['shank_angle']
            d[change_plus_minus] = - d[change_plus_minus]
            
        d['rear_shoulder_adduction'] = - d['rear_shoulder_adduction']
        d['lead_elbow_flexion_velocity'] = - d['lead_elbow_flexion_velocity']
        
        stride = np.sqrt(((d['lead_heel_3d_x'] - d['rear_heel_3d_x']) ** 2 + (d['lead_heel_3d_z'] - d['rear_heel_3d_z']) ** 2))
        stride_length = 100 * (stride.max() / (height / 100))
        
        d['to_frame'] = to_frame
        d['kh_frame'] = kh_frame
        d['fc_frame'] = fc_frame
        d['bc_frame'] = bc_frame

        d['pelvis_rotation_initial'] = pel_rot_initial
        d['time'] = t
        d['weight'] = weight
        d['height'] = height
        d['stride_length'] = stride_length
        d['ball_velocity'] = ball_vel
        d['launch_angle'] = launch_ang
        d['ball_distance'] = ball_dis
        d.to_csv(f"data/{day}/process/{name}_{trial}.csv",index=False)

data/20240213/uplift_data_export_20240213113022.csv


100%|██████████| 2/2 [00:00<00:00, 17.31it/s]


data/20240213/uplift_data_export_20240213131339.csv


100%|██████████| 3/3 [00:00<00:00, 14.93it/s]


data/20240213/uplift_data_export_20240213132832.csv


...

100%|██████████| 2/2 [00:00<00:00, 17.05it/s]


data/20240213/uplift_data_export_20240214012305.csv


100%|██████████| 2/2 [00:00<00:00, 12.15it/s]


data/20240213/uplift_data_export_20240214012403.csv


100%|██████████| 1/1 [00:00<00:00, 14.91it/s]
